https://otexts.org/fpp2/intro.html  
https://machinelearningmastery.com/time-series-forecast-study-python-annual-water-usage-baltimore/  
also: https://machinelearningmastery.com/time-series-forecast-case-study-python-monthly-armed-robberies-boston/

Use this command to convert the notebook into a slideshow that can be presented:


 jupyter nbconvert --to slides --ServePostProcessor.port=8910 --ServePostProcessor.ip='0.0.0.0' --post serve /home/churtado/notebooks/churtado/smmt/SMMT.ipynb  
   
Access the slides here:  
http://192.168.1.123:8910/SMMT.slides.html#/

In [1]:
import plotly
plotly.tools.set_credentials_file(username='churtado', api_key='iaMRV6ydU9Ove5Yfy0R7')

In [2]:
%load_ext sparkmagic.magics

http://192.168.1.123:8998/

In [3]:
%manage_spark

TWFnaWNzQ29udHJvbGxlcldpZGdldChjaGlsZHJlbj0oVGFiKGNoaWxkcmVuPShNYW5hZ2VTZXNzaW9uV2lkZ2V0KGNoaWxkcmVuPShIVE1MKHZhbHVlPXUnPGJyLz4nKSwgSFRNTCh2YWx1ZT3igKY=


Added endpoint http://192.168.1.123:8998/
Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
11,None,spark,idle,,,✔


SparkSession available as 'spark'.


In [10]:
%%spark
val sqlContext = new org.apache.spark.sql.hive.HiveContext(sc)

val df = sqlContext.read.format("com.crealytics.spark.excel").option("sheetName", "pc").option("useHeader", "true").option("inferSchema", "false").load("/home/churtado/notebooks/data/smmt.xls")
df.createTempView("pc")

sqlContext: org.apache.spark.sql.hive.HiveContext = org.apache.spark.sql.hive.HiveContext@674059af
df: org.apache.spark.sql.DataFrame = [date: string, passenger_cars: string]


In [11]:
%%spark -c sql
SHOW TABLES

,database,tableName,isTemporary
0,default,cleaned_taxes,False
1,,pc,True


In [12]:
%%spark -c sql
SELECT * from pc limit 3

,date,passenger_cars
0,2018-06-30,234945
1,2018-05-31,192649
2,2018-04-30,167911


In [13]:
%%spark -c sql
DROP TABLE IF EXISTS passenger_cars

""


In [14]:
%%spark -c sql 

CREATE TABLE passenger_cars AS
SELECT 
 date, 
 passenger_cars
FROM pc

""


In [15]:
%%spark -c sql -q -o df_passenger_cars
SELECT * from pc

ARIMA Forecast using SSMT data
  
* 2 columns: total passenger cars sold, and date
* From january 2013 until june 2018
* Monthly level (last day of each month)

When decomposing a time series, there are 2 components to look out for:  
  
Systematic: Components of the time series that have consistency or recurrence and can be described and modeled.  
Non-Systematic: Components of the time series that cannot be directly modeled.  

Concretely, the components are:  
  
    Level: The average value in the series.  
    Trend: The increasing or decreasing value in the series.  
    Seasonality: The repeating short-term cycle in the series.  
    Noise: The random variation in the series.  

Let's try to decompose it. First set up our dataframe

In [16]:
%matplotlib inline

import pandas as pd
from statsmodels.tsa.seasonal import seasonal_decompose

df_passenger_cars_decomp = df_passenger_cars
df_passenger_cars_decomp['date'] = pd.to_datetime(df_passenger_cars_decomp['date'])
df_passenger_cars_decomp = df_passenger_cars_decomp.set_index('date')

In [17]:
df_passenger_cars_decomp.head(5)

,passenger_cars
date,
2018-06-30,234945
2018-05-31,192649
2018-04-30,167911
2018-03-31,474069
2018-02-28,80805


We're looking at this breakdown assuming an additive model first:

In [18]:
from statsmodels.tsa.seasonal import seasonal_decompose

result_additive = seasonal_decompose(df_passenger_cars_decomp, model='additive')

We'll take the original values from our statsmodel decomp and plot them

In [19]:
import plotly.plotly as py
import plotly.graph_objs as go

trace1_observed = go.Scatter(
    x = result_additive.observed.index,
    y = result_additive.observed.values,
    xaxis='x',
    yaxis='y',
    name='trend'
)
data_observed = [trace1_observed]
layout_observed = go.Layout(
    autosize=False,
    width=800,
    height=500,
    xaxis=dict(
        domain=[0, 1]
    ),
    yaxis=dict(
        domain=[0, 1]
    )
)

In [20]:
fig_data = go.Figure(data=data_observed, layout=layout_observed)
py.iplot(fig_data, filename='observed')

In [21]:
import plotly.plotly as py
import plotly.graph_objs as go

trace1_additive = go.Scatter(
    x = result_additive.trend.index,
    y = result_additive.trend.values,
    xaxis='x3',
    yaxis='y3',
    name='trend'
)
trace2_additive = go.Scatter(
    x = result_additive.seasonal.index,
    y = result_additive.seasonal.values,
    xaxis='x2',
    yaxis='y2',
    name='seasonal'
)
trace3_additive = go.Scatter(
    x = result_additive.resid.index,
    y = result_additive.resid.values,
    xaxis='x',
    yaxis='y',
    name='residual'
)
trace4_additive = go.Scatter(
    x = result_additive.observed.index,
    y = result_additive.observed.values,
    xaxis='x4',
    yaxis='y4',
    name='observed'
)
data_additive = [trace4_additive, trace1_additive, trace2_additive, trace3_additive]
layout_additive = go.Layout(
    autosize=False,
    width=800,
    height=500,
    xaxis=dict(
        domain=[0, 1]
    ),
    yaxis=dict(
        domain=[0, 0.23]
    ),
    xaxis2=dict(
        domain=[0, 1],
        anchor='x2'
    ),
    yaxis2=dict(
        domain=[0.26, 0.48],
        anchor='y2'
    ),
    xaxis3=dict(
        domain=[0, 1],
        anchor='y3'
    ),
    yaxis3=dict(
        domain=[0.52, 0.73]
    ),
    xaxis4=dict(
        domain=[0, 1],
        anchor='y4'
    ),
    yaxis4=dict(
        domain=[0.76, 1],
        anchor='x4'
    )
)


For reference we'll decompose the dataset using a naive decomposition

In [22]:
fig_additive = go.Figure(data=data_additive, layout=layout_additive)
py.iplot(fig_additive, filename='additive-subplots')

Now we'll decompose using a naive multiplicative model  
  
However, as mentioned before, an additive model may be more appropriate.

In [23]:
from statsmodels.tsa.seasonal import seasonal_decompose

result_mult = seasonal_decompose(df_passenger_cars_decomp, model='multiplicative')

In [24]:
import plotly.plotly as py
import plotly.graph_objs as go

trace1_mult = go.Scatter(
    x = result_mult.trend.index,
    y = result_mult.trend.values,
    xaxis='x3',
    yaxis='y3',
    name='trend'
)
trace2_mult = go.Scatter(
    x = result_mult.seasonal.index,
    y = result_mult.seasonal.values,
    xaxis='x2',
    yaxis='y2',
    name='seasonal'
)
trace3_mult = go.Scatter(
    x = result_mult.resid.index,
    y = result_mult.resid.values,
    xaxis='x',
    yaxis='y',
    name='residual'
)
trace4_mult = go.Scatter(
    x = result_mult.observed.index,
    y = result_mult.observed.values,
    xaxis='x4',
    yaxis='y4',
    name='observed'
)
data_mult = [trace4_mult, trace1_mult, trace2_mult, trace3_mult]
layout_mult = go.Layout(
    autosize=False,
    width=800,
    height=500,
    xaxis=dict(
        domain=[0, 1]
    ),
    yaxis=dict(
        domain=[0, 0.23]
    ),
    xaxis2=dict(
        domain=[0, 1],
        anchor='x2'
    ),
    yaxis2=dict(
        domain=[0.26, 0.48],
        anchor='y2'
    ),
    xaxis3=dict(
        domain=[0, 1],
        anchor='y3'
    ),
    yaxis3=dict(
        domain=[0.52, 0.73]
    ),
    xaxis4=dict(
        domain=[0, 1],
        anchor='y4'
    ),
    yaxis4=dict(
        domain=[0.76, 1],
        anchor='x4'
    )
)

In [25]:
fig_mult = go.Figure(data=data_mult, layout=layout_mult)
py.iplot(fig_mult, filename='multiplicative-subplots')

Use of a naive model is generally discouraged, so we've used STL decomposition

We'll now present STL decomposition (Seasonal and Trend decomposition using Loess)  
  
For more information see https://otexts.org/fpp2/stl.html  
The STL decomp package used is here: https://github.com/jrmontag/STLDecompose

STL decomp requires no missing values, so interpolation may be required. Fortunately, there are none in this dataset

In [26]:
from stldecompose import decompose, forecast
from stldecompose.forecast_funcs import (naive, drift, mean, seasonal_naive)

In [27]:
stl = decompose(df_passenger_cars_decomp, period=12)

In [28]:
import plotly.plotly as py
import plotly.graph_objs as go

trace1_stl = go.Scatter(
    x = stl.trend.index,
    y = stl.trend.values,
    xaxis='x3',
    yaxis='y3',
    name='trend'
)
trace2_stl = go.Scatter(
    x = stl.seasonal.index,
    y = stl.seasonal.values,
    xaxis='x2',
    yaxis='y2',
    name='seasonal'
)
trace3_stl = go.Scatter(
    x = stl.resid.index,
    y = stl.resid.values,
    xaxis='x',
    yaxis='y',
    name='residual'
)
trace4_stl = go.Scatter(
    x = stl.observed.index,
    y = stl.observed.values,
    xaxis='x4',
    yaxis='y4',
    name='observed'
)
data_stl = [trace4_stl, trace1_stl, trace2_stl, trace3_stl]
layout_stl = go.Layout(
    autosize=False,
    width=800,
    height=500,
    xaxis=dict(
        domain=[0, 1]
    ),
    yaxis=dict(
        domain=[0, 0.23]
    ),
    xaxis2=dict(
        domain=[0, 1],
        anchor='x2'
    ),
    yaxis2=dict(
        domain=[0.26, 0.48],
        anchor='y2'
    ),
    xaxis3=dict(
        domain=[0, 1],
        anchor='y3'
    ),
    yaxis3=dict(
        domain=[0.52, 0.73]
    ),
    xaxis4=dict(
        domain=[0, 1],
        anchor='y4'
    ),
    yaxis4=dict(
        domain=[0.76, 1],
        anchor='x4'
    )
)

In [29]:
fig_stl = go.Figure(data=data_stl, layout=layout_stl)
py.iplot(fig_stl, filename='stl-decomp')

* Strong seasonal component  
* Neither frequency nor amplitude change dramatically over time.  
* Trend is not linear, but this behavior could be part of a larger cycle, difficult to tell without more data  
* Trend doesn't seem to be multiplicative, so an additive model may be appropriate for decomposition and forecasting  
* Magnitude of sales seems quite stable over the time period for which there is data
* Overall, there are patterns in the data that indicate a forecast model is possible

In [30]:
import plotly.plotly as py
import plotly.graph_objs as go

month = ['January', 'February', 'March', 'April', 'May', 'June', 'July',
         'August', 'September', 'October', 'November', 'December']

# Create and style traces
trace0_seasonal = go.Scatter(
    x = month,
    y = df_passenger_cars_decomp.loc['2013'].values,
    name = '2013',
    #line = dict(color = ('rgb(22, 96, 167)'))
)
trace1_seasonal = go.Scatter(
    x = month,
    y = df_passenger_cars_decomp.loc['2014'].values,
    name = '2014',
    #line = dict(color = ('rgb(22, 96, 167)'))
)
trace2_seasonal = go.Scatter(
    x = month,
    y = df_passenger_cars_decomp.loc['2015'].values,
    name = '2015',
    #line = dict(color = ('rgb(22, 96, 167)'))
)
trace3_seasonal = go.Scatter(
    x = month,
    y = df_passenger_cars_decomp.loc['2016'].values,
    name = '2016',
    #line = dict(color = ('rgb(22, 96, 167)'))
)
trace4_seasonal = go.Scatter(
    x = month,
    y = df_passenger_cars_decomp.loc['2017'].values,
    name = '2017',
    #line = dict(color = ('rgb(22, 96, 167)'))
)
trace5_seasonal = go.Scatter(
    x = month,
    y = df_passenger_cars_decomp.loc['2018'].values,
    name = '2018',
    #line = dict(color = ('rgb(22, 96, 167)'))
)
data_seasonal = [trace0_seasonal, trace1_seasonal, trace2_seasonal, trace3_seasonal, trace4_seasonal, trace5_seasonal]

# Edit the layout
layout_seasonal = dict(title = 'Seasonal Plot Passenger Cars',
              xaxis = dict(title = 'Month'),
              yaxis = dict(title = 'Passenger Cars Sold'),
              #showlegend=False
              )

fig_seasonal = dict(data=data_seasonal, layout=layout_seasonal)

In [31]:
py.iplot(fig_seasonal, filename='seasonal')

* Seasonal plot shows peaks in April and October, valleys in May and November  
* Slight uptick in July and December

In [32]:
import plotly.plotly as py
import plotly.graph_objs as go

month = ['January', 'February', 'March', 'April', 'May', 'June', 'July',
         'August', 'September', 'October', 'November', 'December']

data = [
    go.Scatterpolar(
      r = df_passenger_cars_decomp.loc['2013'].values,
      theta = month,
      name = '2013'
    ),
    go.Scatterpolar(
      r = df_passenger_cars_decomp.loc['2014'].values,
      theta = month,
      name = '2014'
    ),
    go.Scatterpolar(
      r = df_passenger_cars_decomp.loc['2015'].values,
      theta = month,
      name = '2015'
    ),
    go.Scatterpolar(
      r = df_passenger_cars_decomp.loc['2016'].values,
      theta = month,
      name = '2016'
    ),
    go.Scatterpolar(
      r = df_passenger_cars_decomp.loc['2017'].values,
      theta = month,
      name = '2017'
    ),
    go.Scatterpolar(
      r = df_passenger_cars_decomp.loc['2018'].values,
      theta = month,
      name = '2018'
    )
]

layout = go.Layout(
  polar = dict(
    radialaxis = dict(
      visible = False,
      range = [0, 600000]
    )
  ),
  showlegend = False
)

fig = go.Figure(data=data, layout=layout)

In [33]:
py.iplot(fig, filename = "seasonal-polar")

Radial plot

In [34]:
import plotly.plotly as py
import plotly.graph_objs as go

month_seasonal_sub = [2013,2014,2015,2016,2017,2018]

trace1 = go.Scatter(
    x=month_seasonal_sub,
    y=df_passenger_cars_decomp.iloc[::-1][df_passenger_cars_decomp.index.month==1].values,
    name = 'January'
)
trace2 = go.Scatter(
    x=month_seasonal_sub,
    y=df_passenger_cars_decomp.iloc[::-1][df_passenger_cars_decomp.index.month==2].values,
    xaxis='x2',
    yaxis='y2',
    name = 'February'
)
trace3 = go.Scatter(
    x=month_seasonal_sub,
    y=df_passenger_cars_decomp.iloc[::-1][df_passenger_cars_decomp.index.month==3].values,
    xaxis='x3',
    yaxis='y3',
    name = 'March'
)
trace4 = go.Scatter(
    x=month_seasonal_sub,
    y=df_passenger_cars_decomp.iloc[::-1][df_passenger_cars_decomp.index.month==4].values,
    xaxis='x4',
    yaxis='y4',
    name = 'April'
)
data = [trace1, trace2, trace3, trace4]
layout = go.Layout(
    xaxis=dict(
        domain=[0, 0.45]
    ),
    yaxis=dict(
        domain=[0, 0.45]
    ),
    xaxis2=dict(
        domain=[0.55, 1]
    ),
    xaxis3=dict(
        domain=[0, 0.45],
        anchor='y3'
    ),
    xaxis4=dict(
        domain=[0.55, 1],
        anchor='y4'
    ),
    yaxis2=dict(
        domain=[0, 0.45],
        anchor='x2'
    ),
    yaxis3=dict(
        domain=[0.55, 1]
    ),
    yaxis4=dict(
        domain=[0.55, 1],
        anchor='x4'
    )
)
fig = go.Figure(data=data, layout=layout)

In [35]:
py.iplot(fig, filename='sub-season-t1')

In [36]:
import plotly.plotly as py
import plotly.graph_objs as go

trace1 = go.Scatter(
    x=month_seasonal_sub,
    y=df_passenger_cars_decomp.iloc[::-1][df_passenger_cars_decomp.index.month==5].values,
    name = 'May'
)
trace2 = go.Scatter(
    x=month_seasonal_sub,
    y=df_passenger_cars_decomp.iloc[::-1][df_passenger_cars_decomp.index.month==6].values,
    xaxis='x2',
    yaxis='y2',
    name = 'June'
)
trace3 = go.Scatter(
    x=month_seasonal_sub,
    y=df_passenger_cars_decomp.iloc[::-1][df_passenger_cars_decomp.index.month==7].values,
    xaxis='x3',
    yaxis='y3',
    name = 'July'
)
trace4 = go.Scatter(
    x=month_seasonal_sub,
    y=df_passenger_cars_decomp.iloc[::-1][df_passenger_cars_decomp.index.month==8].values,
    xaxis='x4',
    yaxis='y4',
    name = 'August'
)
data = [trace1, trace2, trace3, trace4]
layout = go.Layout(
    xaxis=dict(
        domain=[0, 0.45]
    ),
    yaxis=dict(
        domain=[0, 0.45]
    ),
    xaxis2=dict(
        domain=[0.55, 1]
    ),
    xaxis3=dict(
        domain=[0, 0.45],
        anchor='y3'
    ),
    xaxis4=dict(
        domain=[0.55, 1],
        anchor='y4'
    ),
    yaxis2=dict(
        domain=[0, 0.45],
        anchor='x2'
    ),
    yaxis3=dict(
        domain=[0.55, 1]
    ),
    yaxis4=dict(
        domain=[0.55, 1],
        anchor='x4'
    )
)
fig = go.Figure(data=data, layout=layout)

In [37]:
py.iplot(fig, filename='sub-season-t2')

In [38]:
import plotly.plotly as py
import plotly.graph_objs as go



trace1 = go.Scatter(
    x=month_seasonal_sub,
    y=df_passenger_cars_decomp.iloc[::-1][df_passenger_cars_decomp.index.month==9].values,
    name = 'September'
)
trace2 = go.Scatter(
    x=month_seasonal_sub,
    y=df_passenger_cars_decomp.iloc[::-1][df_passenger_cars_decomp.index.month==10].values,
    xaxis='x2',
    yaxis='y2',
    name = 'October'
)
trace3 = go.Scatter(
    x=month_seasonal_sub,
    y=df_passenger_cars_decomp.iloc[::-1][df_passenger_cars_decomp.index.month==11].values,
    xaxis='x3',
    yaxis='y3',
    name = 'November'
)
trace4 = go.Scatter(
    x=month_seasonal_sub,
    y=df_passenger_cars_decomp.iloc[::-1][df_passenger_cars_decomp.index.month==12].values,
    xaxis='x4',
    yaxis='y4',
    name = 'December'
)
data = [trace1, trace2, trace3, trace4]
layout = go.Layout(
    xaxis=dict(
        domain=[0, 0.45]
    ),
    yaxis=dict(
        domain=[0, 0.45]
    ),
    xaxis2=dict(
        domain=[0.55, 1]
    ),
    xaxis3=dict(
        domain=[0, 0.45],
        anchor='y3'
    ),
    xaxis4=dict(
        domain=[0.55, 1],
        anchor='y4'
    ),
    yaxis2=dict(
        domain=[0, 0.45],
        anchor='x2'
    ),
    yaxis3=dict(
        domain=[0.55, 1]
    ),
    yaxis4=dict(
        domain=[0.55, 1],
        anchor='x4'
    )
)
fig = go.Figure(data=data, layout=layout)

In [39]:
py.iplot(fig, filename='sub-season-t3')

This is a good starting point for understanding autocorrelation plots:  
https://stats.stackexchange.com/questions/101467/how-to-interpret-autocorrelation
  
Quoting from the article:  
...Those plots are showing you the correlation of the series with itself, lagged by x time units. So imagine taking your time series of length T, copying it, and deleting the first observation of copy#1 and the last observation of copy#2. Now you have two series of length T−1 for which you calculate a correlation coefficient. This is the value of of the vertical axis at x=1 in your plots. It represents the correlation of the series lagged by one time unit. You go on and do this for all possible time lags x

and this defines the plot.

The answer to your question of what is needed to report a pattern is dependent on what pattern you would like to report. But quantitatively speaking, you have exactly what I just described: the correlation coefficient at different lags of the series. You can extract these numerical values by issuing the command...

The correlation coefficient is defined as:  
https://www.dummies.com/education/math/statistics/how-to-interpret-a-correlation-coefficient-r/

We will now try to run a naive forecast on the data

Naive forecasts just take the last value and extend it

We'll start the forecasting exercise by first creating a test harness. For this we'll split the data set into a training and validation set, and we'll develop models that will give a baseline for effective forecasting.  
  
In order to do this, we'll split out the last 10 values from the data set and use that to validate the model. If our model can predict these last 10 values well, then it's a good model.

In [42]:
series = df_passenger_cars_decomp
split_point = len(series) - 10
dataset, validation = series[0:split_point], series[split_point:]

print('Dataset %d, Validation %d' % (len(dataset), len(validation)))

Dataset 56, Validation 10


We will evaluate the performance of predictions using the root mean squared error (RMSE). This will give more weight to predictions that are grossly wrong and will have the same units as the original data. We'll be doing what's called a naive forecast. We'll take the last observed value and use that as our next forecasted value, or prediction.

Candidate models will be evaluated using walk-forward validation.

This is because a rolling-forecast type model is required from the problem definition. This is where one-step forecasts are needed given all available data.

The walk-forward validation will work as follows:

    * The first 50% of the dataset will be held back to train the model.
    * The remaining 50% of the dataset will be iterated and test the model.
    * For each step in the test dataset:
        * A model will be trained.
        * A one-step prediction made and the prediction stored for later evaluation.
        * The actual observation from the test dataset will be added to the training dataset for the next iteration.
    * The predictions made during the enumeration of the test dataset will be evaluated and an RMSE score reported.

Given the small size of the data, we will allow a model to be re-trained given all available data prior to each prediction.

In [43]:
# prepare data
X = series.values
X = X.astype('float32')
train_size = int(len(X) * 0.50)
train, test = X[0:train_size], X[train_size:]

In [45]:
from pandas import Series
from sklearn.metrics import mean_squared_error
from math import sqrt

# walk-forward validation
history = [x for x in train] # this is our training dataset
predictions = list()
for i in range(len(test)):
    # predict
    yhat = history[-1] # get the last value from the training data
    predictions.append(yhat) # append that last value from the training data as the first prediction
    # observation
    obs = test[i] # this is the actual value vs the forecasted that was appended to the predictions list
    history.append(obs) # append the observed value in the history
    print('>Predicted=%.3f, Expected=%3.f' % (yhat, obs))

>Predicted=177664.000, Expected=462517
>Predicted=462517.000, Expected=79060
>Predicted=79060.000, Expected=178420
>Predicted=178420.000, Expected=257817
>Predicted=257817.000, Expected=198706
>Predicted=198706.000, Expected=185778
>Predicted=185778.000, Expected=492774
>Predicted=492774.000, Expected=76958
>Predicted=76958.000, Expected=164856
>Predicted=164856.000, Expected=166198
>Predicted=166198.000, Expected=172327
>Predicted=172327.000, Expected=179714
>Predicted=179714.000, Expected=425861
>Predicted=425861.000, Expected=72163
>Predicted=72163.000, Expected=172907
>Predicted=172907.000, Expected=228291
>Predicted=228291.000, Expected=194032
>Predicted=194032.000, Expected=176820
>Predicted=176820.000, Expected=464824
>Predicted=464824.000, Expected=68736
>Predicted=68736.000, Expected=154562
>Predicted=154562.000, Expected=152918
>Predicted=152918.000, Expected=159581
>Predicted=159581.000, Expected=157314
>Predicted=157314.000, Expected=403136
>Predicted=403136.000, Expected=6

In [46]:
# report performance
mse = mean_squared_error(test, predictions)
rmse = sqrt(mse)
print('RMSE: %.3f' % rmse)

RMSE: 200408.425


Our root mean square error is roughly 200k units (passenger cars)

Let's generate some summary statistics

In [47]:
print(series.describe())

       passenger_cars
count       66.000000
mean    210940.484848
std     123740.275669
min      65937.000000
25%     157533.500000
50%     177843.000000
75%     212114.000000
max     562337.000000


* The number of observations matches the expectation, so we're handling the data correctly
* Meas is about 200K, which is our level in the series
* Std dev is roughly half the mean, so we can expect a reasonable amount of oscillation around the mean

Let's look at a density plot of the observed values in our series

In [48]:
import plotly.plotly as py
import plotly.graph_objs as go

import numpy as np

x = series.values
data = [go.Histogram(x=x)]

In [49]:
py.iplot(data, filename='basic histogram')

Values are roughly centered around the mean, and long right tail

Let's look at a normalized histogram

In [50]:
import plotly.plotly as py
import plotly.graph_objs as go

import numpy as np

data = [go.Histogram(x=series.values, histnorm='probability')]

In [52]:
# plotly free only allows 25 images. fuck
plotly.tools.set_credentials_file(username='churtado_mazepoint', api_key='u7rxs8F6GDdGs2P5E2Tz')

In [53]:
py.iplot(data, filename='normalized_histogram')

* Roughly half the values hover around 200k mark

Let's look at a density plot. Because of a defficiency in numpy, we're sending large numbers that internally are getting converted to NaN's or INF's. We'll apply standardization to the series and use that for the density plot.

See: https://stackoverflow.com/questions/14470012/scipy-optimize-curvefit-array-must-not-contain-infs-or-nans/36390482#36390482

In [97]:
import plotly.plotly as py
import plotly.figure_factory as ff

import numpy as np

x = series.values # getting the values
x = (x - np.mean(x, axis=0)) / np.std(x, axis=0) # normalizing the values
x = np.transpose(x)

hist_data = x

group_labels = ['distplot']

fig_dist = ff.create_distplot(hist_data, group_labels)

In [98]:
py.iplot(fig_dist, filename='Basic Distplot')

* Again, long right tail
* Perhaps a double Gaussian distribution

In [72]:
import plotly.plotly as py
import plotly.graph_objs as go

x_data = [2013,2014,2015,2016,2017,2018]

y0 = series["2013"].values
y1 = series["2014"].values
y2 = series["2015"].values
y3 = series["2016"].values
y4 = series["2017"].values
y5 = series["2018"].values

y_data = [y0,y1,y2,y3,y4,y5]

colors = ['rgba(93, 164, 214, 0.5)', 
          'rgba(255, 144, 14, 0.5)', 
          'rgba(44, 160, 101, 0.5)', 
          'rgba(255, 65, 54, 0.5)', 
          'rgba(207, 114, 255, 0.5)', 
          'rgba(127, 96, 0, 0.5)']

traces = []

for xd, yd, cls in zip(x_data, y_data, colors):
        traces.append(go.Box(
            y=yd,
            name=xd,
            boxpoints='all',
            jitter=0.5,
            whiskerwidth=0.2,
            fillcolor=cls,
            marker=dict(
                size=2,
            ),
            line=dict(width=1),
        ))

layout = go.Layout(
    title='Box Plot',
    yaxis=dict(
        autorange=True,
        showgrid=True,
        zeroline=False,
        #dtick=5,
        gridcolor='rgb(255, 255, 255)',
        gridwidth=1,
        zerolinecolor='rgb(255, 255, 255)',
        zerolinewidth=2,
    ),
    margin=dict(
        l=40,
        r=30,
        b=80,
        t=100,
    ),
    showlegend=False
)

fig_box = go.Figure(data=traces, layout=layout)

In [73]:
py.iplot(fig_box)

* Outliers on top are probably the peaks in the trend (new license registrations)
* Troughs would correspond to the valleys
* Median trend is similar to decomposition trend
* Spread is quite regular, except for last 2 years (but could be due to lack of data)

We will now try to create an ARIMA model using this data to forecast PC numbers into the future.  


ARIMA is an acronym that stands for AutoRegressive Integrated Moving Average. It is a generalization of the simpler AutoRegressive Moving Average and adds the notion of integration.

This acronym is descriptive, capturing the key aspects of the model itself. Briefly, they are:

    AR: Autoregression. A model that uses the dependent relationship between an observation and some number of lagged observations.
    I: Integrated. The use of differencing of raw observations (e.g. subtracting an observation from an observation at the previous time step) in order to make the time series stationary.
    MA: Moving Average. A model that uses the dependency between an observation and a residual error from a moving average model applied to lagged observations.


The parameters of the ARIMA model are defined as follows:

    p: The number of lag observations included in the model, also called the lag order.
    d: The number of times that the raw observations are differenced, also called the degree of differencing.
    q: The size of the moving average window, also called the order of moving average.


Before we run the ARIMA model on the data, the series has to be tested for stationarity. Although from the decomposition, I believe it's pretty clear it's not a stationary process (the trend varies over time, altering the mean)

To put it in simpler terms, it has to have a constant mean and variance. And to be clear, stationarity is a property of a process, not a time series.

See here: https://en.wikipedia.org/wiki/Stationary_process  
See also: https://stats.stackexchange.com/questions/9951/intuitive-explanation-of-stationarity

There are several ways to test for stationarity. First we'll apply a simple method.
  
What we'll do is split the series into 2, and calculate the mean and variance of each group of numbers and compare the values. It's a way of seeing if the avg and variance stay constant in the long run. 

* The % differences seem to be small (3% for mean and 8% for variance)  
* But in the case of the variance it's 15 billion vs 14 billion  
* The ARIMA model failed for stationarity when I eventually tried to run it

In [52]:
from pandas import Series

series = df_passenger_cars_decomp
X = series.values
split = 33
X1, X2 = X[0:split], X[split:]
mean1, mean2 = X1.mean(), X2.mean()
var1, var2 = X1.var(), X2.var()

In [53]:
print('mean1=%f, mean2=%f' % (mean1, mean2))
print('variance1=%f, variance2=%f' % (var1, var2))

print('meanDif_pct=%f, varDif_pct=%f' % ((mean1-mean2)/mean1, (var1-var2)/var1))

mean1=214667.090909, mean2=207213.878788
variance1=15697704021.719006, variance2=14433842867.924702
meanDif_pct=0.034720, varDif_pct=0.080512


* We've established that the series is non-stationary
* We can make the series stationary by first differencing the series 
* We'll use a statistical test to confirm that the result is stationary.

In [79]:
from statsmodels.tsa.stattools import adfuller
 
# create a differe
def difference(dataset):
    diff = list()
    for i in range(1, len(dataset)):
        value = dataset[i] - dataset[i - 1]
        diff.append(value)
    return Series(diff)
 
X = series.values
X = X.astype('float32')

# difference data
stationary = difference(X)
stationary.index = series.index[1:]

# check if stationary
result = adfuller(stationary)

In [80]:
print('ADF Statistic: %f' % result[0])
print('p-value: %f' % result[1])
print('Critical Values:')
for key, value in result[4].items():
    print('\t%s: %.3f' % (key, value))

ADF Statistic: -11.324056
p-value: 0.000000
Critical Values:
	5%: -2.917
	1%: -3.558
	10%: -2.596


Running the example outputs the result of a statistical significance test of whether the differenced series is stationary. Specifically, the augmented Dickey-Fuller test.

The results show that the test statistic value is smaller than the critical value at 1%. This suggests that we can reject the null hypothesis with a significance level of less than 1% (i.e. a low probability that the result is a statistical fluke).

Rejecting the null hypothesis means that the process has no unit root, and in turn that the time series is stationary or does not have time-dependent structure.

https://en.wikipedia.org/wiki/Unit_root

In [110]:
# plot differenced data
stationary

import plotly.plotly as py
import plotly.graph_objs as go
import math 

from statsmodels.tsa.stattools import acf, pacf

# Create and style traces
trace0_stationary = go.Scatter(
    x = stationary.index,
    y = stationary.values,
    name = 'stationary',
    #line = dict(color = ('rgb(22, 96, 167)'))
)
data_stationary = [trace0_stationary]

# Edit the layout
layout_stationary = dict(title = 'Stationary Series',
              xaxis = dict(title = ''),
              yaxis = dict(title = ''),
              showlegend=False
              )

fig_stationary = dict(data=data_stationary, layout=layout_stationary)

In [112]:
py.iplot(fig_stationary, filename='stationary')

We now have a stationary series. In order to determine the parameters for the ARIMA forecast

In [87]:
import plotly.plotly as py
import plotly.graph_objs as go
import math 

from statsmodels.tsa.stattools import acf, pacf

acf =  acf(df_passenger_cars_decomp)
pacf = pacf(df_passenger_cars_decomp)

lags = list(range(0,acf.size+1))

upper = [2/math.sqrt(acf.size)] * acf.size
lower = [-2/math.sqrt(acf.size)] * acf.size

In [88]:
# Create and style traces
trace0_acf = go.Scatter(
    x = lags,
    y = acf,
    name = 'acf',
    #line = dict(color = ('rgb(22, 96, 167)'))
)
    
trace1_acf = go.Scatter(
    x = lags,
    y = upper,
    name = 'upper',
    line = dict(
        color = ('rgb(22, 96, 167)'),
        width = 2,
        dash = 'dash'
    )
)
trace2_acf = go.Scatter(
    x = lags,
    y = lower,
    name = 'lower',
    line = dict(
        color = ('rgb(22, 96, 167)'),
        width = 2,
        dash = 'dash'
    )
)

data_acf = [trace0_acf, trace1_acf, trace2_acf]

# Edit the layout
layout_acf = dict(title = 'ACF Plot',
              xaxis = dict(title = 'Lag'),
              yaxis = dict(title = 'Correlation Coefficient'),
              #showlegend=False
              )

fig_acf = dict(data=data_acf, layout=layout_acf)

In [89]:
py.iplot(fig_acf, filename='acf')

* Highest correlation coefficient observed is at lag 6. The cycle of peaks and valleys tend to repeat at this frequency
* This is due to the seasonal pattern in the data: the peaks tend to be 6 months apart and the troughs tend to be 4 and then two months apart.


In [108]:
# Create and style traces
trace0_pacf = go.Scatter(
    x = lags,
    y = pacf,
    name = 'pacf',
    #line = dict(color = ('rgb(22, 96, 167)'))
)
    
trace1_pacf = go.Scatter(
    x = lags,
    y = upper,
    name = 'upper',
    line = dict(
        color = ('rgb(22, 96, 167)'),
        width = 2,
        dash = 'dash'
    )
)
trace2_pacf = go.Scatter(
    x = lags,
    y = lower,
    name = 'lower',
    line = dict(
        color = ('rgb(22, 96, 167)'),
        width = 2,
        dash = 'dash'
    )
)

data_pacf = [trace0_pacf]

# Edit the layout
layout_pacf = dict(title = 'PACF Plot',
              xaxis = dict(title = 'Lag'),
              yaxis = dict(title = 'Partial Correlation Coefficient'),
              #showlegend=False
              )

fig_pacf = dict(data=data_pacf, layout=layout_pacf)

In [109]:
py.iplot(fig_pacf, filename='pacf')

PACF shows no significant lags

* Based on the ACF plot, 6 would be a good lag (p) for an ARIMA forecast model
* From the stationary plot, 1 order differencing may be effective (d)
* We'll experiment with the size of the moving avg window (q) starting with 0

In [168]:
from pandas import Series
from sklearn.metrics import mean_squared_error
from statsmodels.tsa.arima_model import ARIMA
from math import sqrt

# prepare data
X = series.values
X = X.astype('float32')
train_size = int(len(X) * 0.50)
train, test = X[0:train_size], X[train_size:]
time = series.index[0:int(len(X) * 0.50)]

# walk-forward validation
history = [x for x in train]
predictions = list()

for i in range(len(test)):
    # predict
    model = ARIMA(history, order=(0,1,0))
    model_fit = model.fit(disp=0)
    yhat = model_fit.forecast()[0]
    predictions.append(yhat)
    # observation
    obs = test[i]
    history.append(obs)
    print('>Predicted=%.3f, Expected=%3.f' % (yhat, obs))


test = test[:len(test)-1]
predictions = predictions[1:]
    
# Create and style traces
trace0_arima = go.Scatter(
    x = time,
    y = test,
    name = 'test',
    #line = dict(color = ('rgb(22, 96, 167)'))
)
    
trace1_arima = go.Scatter(
    x = time,
    y = predictions,
    name = 'predictions',
    line = dict(
        #color = ('rgb(22, 96, 167)'),
        width = 2,
        dash = 'dash'
    )
)

data_arima = [trace0_arima, trace1_arima]

# Edit the layout
layout_arima = dict(title = 'ARIMA Test Plot',
              xaxis = dict(title = ''),
              yaxis = dict(title = ''),
              #showlegend=False
              )

fig_arima = dict(data=data_arima, layout=layout_arima)
    
# report performance
mse = mean_squared_error(test, predictions)
rmse = sqrt(mse)


>Predicted=175873.969, Expected=462517
>Predicted=469413.121, Expected=79060
>Predicted=74475.147, Expected=178420
>Predicted=176805.000, Expected=257817
>Predicted=258452.333, Expected=198706
>Predicted=197726.568, Expected=185778
>Predicted=184484.132, Expected=492774
>Predicted=499385.000, Expected=76958
>Predicted=73008.325, Expected=164856
>Predicted=163146.512, Expected=166198
>Predicted=164561.167, Expected=172327
>Predicted=170870.767, Expected=179714
>Predicted=178458.750, Expected=425861
>Predicted=430103.578, Expected=72163
>Predicted=68624.261, Expected=172907
>Predicted=171587.043, Expected=228291
>Predicted=228152.375, Expected=194032
>Predicted=193197.041, Expected=176820
>Predicted=175657.500, Expected=464824
>Predicted=469331.431, Expected=68736
>Predicted=65539.673, Expected=154562
>Predicted=153045.340, Expected=152918
>Predicted=151398.981, Expected=159581
>Predicted=158210.745, Expected=157314
>Predicted=155927.732, Expected=403136
>Predicted=406086.719, Expected=6

In [169]:
print('RMSE: %.3f' % rmse)
py.iplot(fig_arima, filename='arima')

RMSE: 2797.392
